In [1]:
#!pip install langchain
#!pip install chromadb
#!pip install rank_bm25

In [2]:
from langchain.retrievers import EnsembleRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever
from langchain.schema import Document
import os 

#openai key 
os.environ['OPENAI_API_KEY'] = 'you code'

# Initialize components
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(embedding_function=embeddings)

# docu
documents = [
    Document(page_content="AI and machine learning advancements", metadata={"year": 2023}),
    Document(page_content="Climate change effects on biodiversity", metadata={"year": 2022}),
    Document(page_content="Quantum computing breakthroughs", metadata={"year": 2024}),
]

# Add documents to vectorstore and create BM25 index
vectorstore.add_documents(documents)
bm25_retriever = BM25Retriever.from_documents(documents)

# Create ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[vectorstore.as_retriever(), bm25_retriever],
    weights=[0.5, 0.5]
)

# Define a filter function
def year_filter(doc):
    return doc.metadata["year"] >= 2023

# Perform hybrid search with filter
query = "Recent advancements in technology"
raw_docs = ensemble_retriever.get_relevant_documents(query)
filtered_docs = list(filter(year_filter, raw_docs))

for doc in filtered_docs:
    print(f"Retrieved document: {doc.page_content}")
    print(f"Year: {doc.metadata['year']}")
    print()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


Retrieved document: AI and machine learning advancements
Year: 2023

Retrieved document: Quantum computing breakthroughs
Year: 2024

